In [60]:
import os
import json
import copy
import pandas as pd
import math
import seaborn as sns

FAKE = {
    "mip": False,
    "has_mipgap": None,
    "mipgap": None,
    "runtime_ms": None,
    "status": None,
}


In [61]:
def flatten_dict(d, prefix):
    is_dict = isinstance(d, dict)

    newdict = dict()
    if is_dict:
        for k, v in d.items():
            nk = f"{prefix}.{k}"
            newdict[nk] = copy.deepcopy(v)
    return newdict


def flatten_list(d, prefix):
    is_list = isinstance(d, list)

    newdict = dict()
    if is_list:
        r = []
        for i, v in enumerate(d):
            nk = f"{prefix}[{i}]"
            r.append(flatten_dict(v, nk))
        for dic in r:
            newdict = newdict | dic

    return newdict


In [62]:
FOLDER = "./results"
PREFIX = "esvis-ds"

folders_with_call_file = []

for subdir, dirs, files in os.walk(FOLDER):
    if "call.json" in files and subdir.startswith(os.path.join(FOLDER, PREFIX)):
        folders_with_call_file.append(subdir)

calls = []
for dir in folders_with_call_file:
    callfile = os.path.join(dir, "call.json")
    with open(callfile, "r") as f:
        call = json.load(f)
        call = call | flatten_dict(call["ctx"], "ctx")
        del call["ctx"]
        if "metrics" in call:
            call = call | flatten_list(call["metrics"], "metrics")
            del call["metrics"]

        # also merge the embedding metrics
        embedding_metrics = None
        with open(os.path.join(dir, 'metrics_embedding.json')) as f:
            embedding_metrics = json.load(f)
            call = call | embedding_metrics[0]

        # merge also the additional files
        for ilp_status_file in ["connect", "overlapremoval", "route", 'layout']:
            filename = f"ilp_status_{ilp_status_file}_0.json"
            filepath = os.path.join(dir, filename)
            status_file_exists = os.path.exists(filepath)
            if status_file_exists:
                with open(filepath, "r") as f:
                    status_file_data = flatten_dict(
                        json.load(f), f"ilp_{ilp_status_file}"
                    )
                    call = call | status_file_data
            else:
                call = call | FAKE
        calls.append(call)

df = pd.DataFrame(data=calls)


In [67]:
# post process

df["success"] = df.success.astype("bool")



0      0.000000e+00
1     NaN          
2     NaN          
3      0.000000e+00
4      3.764772e-16
           ...     
402   NaN          
403    0.000000e+00
404    0.000000e+00
405    0.000000e+00
406    2.604192e-16
Name: ilp_connect.mipgap, Length: 407, dtype: float64

In [73]:
df[['success', 'duration_ms', 'ilp_route.status','ctx.route.router']]


,success,duration_ms,ilp_route.status,ctx.route.router
0,True,2.464736e+04,NaN,heuristic
1,False,1.626652e+06,NaN,heuristic
2,False,1.026145e+06,NaN,opt
3,True,9.980867e+03,NaN,heuristic
4,False,1.038877e+06,NaN,opt
...,...,...,...,...
402,False,9.770941e+05,NaN,opt
403,True,5.459388e+04,2.0,opt
404,True,1.259515e+06,2.0,opt
405,True,4.124022e+05,NaN,heuristic
